<a href="https://colab.research.google.com/github/Jg-Nascimento/trading/blob/master/PredictingStockPricesUsingAnLSTM_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[Artigo link 1](https://towardsdatascience.com/lstm-time-series-forecasting-predicting-stock-prices-using-an-lstm-model-6223e9644a2f)

[Artigo link 2](https://data-flair.training/blogs/stock-price-prediction-machine-learning-project-in-python/)

[Artigo link 3](https://randerson112358.medium.com/stock-price-prediction-using-python-machine-learning-e82a039ac2bb)

[Artigo Link 4](https://data-flair.training/blogs/stock-price-prediction-machine-learning-project-in-python/)


#**Previsão de série temporal:**

###prevendo preços de ações usando um modelo LSTM

carregando os dados

In [ ]:
import math
import matplotlib.pyplot as plt
import keras
import pandas as pd
import numpy as np

In [ ]:
from pandas_datareader import data as web
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers import *
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping

In [ ]:
'''#lendo o arquivo csv
df = pd.read_csv('TSLA.csv')
print('Número de Linhas e colunas: ', df.shape)
df.head(5) => original'''

#escolher a ação desejada
#acao = input("Digite a acao desejada: ")
acao = 'tsla'

# criando DataFframe com dados a partir de 1-1-2000
df = web.DataReader(acao, data_source='yahoo', start='01-01-2020')

df.head(5)

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2020-01-02,86.139999,84.342003,84.900002,86.052002,47660500.0,86.052002
2020-01-03,90.800003,87.384003,88.099998,88.601997,88892500.0,88.601997
2020-01-06,90.311996,88.000000,88.094002,90.307999,50665000.0,90.307999
2020-01-07,94.325996,90.671997,92.279999,93.811996,89410500.0,93.811996
2020-01-08,99.697998,93.646004,94.739998,98.428001,155721500.0,98.428001


In [ ]:
#linha acrescentada
#mostra a quantidade de linhas e colunas
df.shape

(248, 6)

In [ ]:
#linha acrescentada
#resumo
df.describe()

,High,Low,Open,Close,Volume,Adj Close
count,248.000000,248.000000,248.000000,248.000000,2.480000e+02,248.000000
mean,289.440557,272.985073,281.427187,282.166766,6.272551e+07,282.166766
std,164.140630,155.549305,160.237832,161.294449,4.606973e+07,161.294449
min,80.972000,70.101997,74.940002,72.244003,6.362400e+06,72.244003
25%,154.403000,142.331501,147.276501,149.528500,2.649180e+07,149.528500
50%,211.516998,199.820000,202.462997,210.486000,5.526680e+07,210.486000
75%,428.184990,407.535004,417.327507,419.784996,8.685888e+07,419.784996
max,695.000000,646.070007,668.900024,695.000000,3.046940e+08,695.000000


A próxima etapa é dividir os dados em conjuntos de treinamento e teste para evitar overfitting 

e poder investigar a capacidade de generalização de nosso modelo. Para saber mais sobre overfitting, 

[leia este artigo](https://towardsdatascience.com/is-your-model-overfitting-or-maybe-underfitting-an-example-using-a-neural-network-in-python-4faf155398d2)

---
## O valor alvo a ser previsto será o valor do preço da ação “Fechar”.

---


In [ ]:
 '''training_set = df.iloc[:800, 1:2].values
 test_set = df.iloc[800:, 1:2].values'''
 
 #acrescentada
 training_set = df.iloc[:230, 1:2].values
 test_set = df.iloc[230:, 1:2].values

É uma boa ideia normalizar os dados antes do ajuste do modelo. 

Isso aumentará o desempenho. Você pode ler mais [aqui](https://towardsdatascience.com/everything-you-need-to-know-about-min-max-normalization-in-python-b79592732b79) para o Min-Max Scaler

Vamos construir os recursos de entrada com intervalo de tempo de 1 dia (intervalo 1):



In [ ]:
#Dimensionamento de Recursos (Feature Scaling)
sc = MinMaxScaler(feature_range=(0, 1))
training_set_scaled = sc.fit_transform(training_set)

#Criação de uma estrutura de dados com 60 etapas de tempo e 1 saída
X_train = []
y_train = []
for i in range(60, 230):
  X_train.append(training_set_scaled[i-60: i, 0])
  y_train.append(training_set_scaled[i, 0])

X_train, y_train = np.array(X_train), np.array(y_train)
X_train = np.reshape(X_train, (X_train.shape[0], 
                               X_train.shape[1], 1))

Agora remodelamos os dados para o seguinte formato (# valores, # etapas de tempo, 

1 saída dimensional).

Agora é hora de construir o modelo. Vamos construir o LSTM com 50 neurônios e 4 camadas 

ocultas. Por fim, atribuiremos 1 neurônio na camada de saída para prever o preço normalizado das ações. 

Usaremos a função de perda MSE e o otimizador de descida gradiente estocástico de 
Adam

In [ ]:
model = Sequential()
#Adicionando a primeira camada LSTM e alguma regularização de Dropout
model.add(LSTM(units = 50, return_sequences = True, 
               input_shape = (X_train.shape[1], 1)))

# Adicionando a segunda camada LSTM e alguma regularização de Dropout
model.add(Dropout(0.2))
model.add(LSTM(units = 50, return_sequences = True))

# Adicionando a terceira camada LSTM e alguma regularização de Dropout
model.add(Dropout(0.2))
model.add(LSTM(units = 50, return_sequences = True))

# Adicionando a quarta camada LSTM e alguma regularização de Dropout
model.add(LSTM(units = 50))
model.add(Dropout(0.2))

# Adicionando a camada de saída
model.add(Dense(units = 1))

# Compilando o RNN
model.compile(optimizer = 'adam', loss = 'mean_squared_error')

# Ajustando o RNN ao conjunto de treinamento
model.fit(X_train, y_train, epochs = 10, batch_size = 32)

Epoch 1/10
6/6 [==============================] - 6s 91ms/step - loss: 0.1829
Epoch 2/10
6/6 [==============================] - 1s 94ms/step - loss: 0.0484
Epoch 3/10
6/6 [==============================] - 1s 99ms/step - loss: 0.0225
Epoch 4/10
6/6 [==============================] - 1s 97ms/step - loss: 0.0136
Epoch 5/10
6/6 [==============================] - 1s 98ms/step - loss: 0.0108
Epoch 6/10
6/6 [==============================] - 1s 97ms/step - loss: 0.0093
Epoch 7/10
6/6 [==============================] - 1s 100ms/step - loss: 0.0070
Epoch 8/10
6/6 [==============================] - 1s 101ms/step - loss: 0.0088
Epoch 9/10
6/6 [==============================] - 1s 120ms/step - loss: 0.0084
Epoch 10/10
6/6 [==============================] - 1s 103ms/step - loss: 0.0076


Prepare os dados de teste (reformule-os)

In [ ]:
#Obtendo o preço de ação previsto para 2017
dataset_train = df.iloc[:230, 1:2]
dataset_test = df.iloc[0:, 1:2]
dataset_total = pd.concat((dataset_train, dataset_test), axis = 0)
inputs = dataset_total[len(dataset_total) - len(dataset_test) - 60:].values
inputs = inputs.reshape(-1,1)
inputs = sc.transform(inputs)

#criando um set de teste
x_test = []

for i in range(60, 230):
  x_test.append(inputs[i-60:i, 0])

x_test = np.array(x_test)
#convertendo em 3d (requerido para processo)
x_test = np.reshape(x_test,(x_test.shape[0],
                            x_test.shape[1], 1))

print(x_test.shape)

(170, 60, 1)


Faça previsões usando o conjunto de teste

In [ ]:
predicted_stock_price = model.predict(x_test)
predicted_stock_price = sc.inverse_transform(predicted_stock_price)

visualisando os dados

In [ ]:
plt.plot(df.loc[226: , 'Date'], dataset_test.values,
         color= 'red', label='Real TESLA Stock Price')
plt.plot(df.loc[226:, 'Date'], predicted_stock_price, 
         color='blue', label='Predicted TESLA Stock Price')

plt.xticks(np.arange(0, 459, 50))
plt.title('TESLA Stock Price Prediction')
plt.xlabel('Time')
plt.ylabel('Tesla Stock Price')
plt.legend()
plt.show()

KeyError: ignored